In [7]:
# --- 0️⃣ Імпорт бібліотек та даних ---
import pandas as pd

# Заміни шлях на свій файл
df = pd.read_csv("data/index_1.csv")

# --- 1️⃣ Попередня обробка ---
df['weekday'] = pd.to_datetime(df['datetime']).dt.day_name()
df['month'] = pd.to_datetime(df['datetime']).dt.month_name()
df['is_weekend'] = df['weekday'].isin(['Saturday', 'Sunday'])
df['hour_of_day'] = pd.to_datetime(df['datetime']).dt.hour
df['time'] = pd.to_datetime(df['datetime']).dt.time

# Функція для визначення пікового періоду
def define_peak(hour):
    if 7 <= hour <= 10:
        return 'Morning Peak'
    elif 12 <= hour <= 14:
        return 'Lunch Peak'
    elif 17 <= hour <= 19:
        return 'Evening Peak'
    else:
        return 'Off-Peak'

df['peak_period'] = df['hour_of_day'].apply(define_peak)
df['day_type'] = df['is_weekend'].apply(lambda x: 'Weekend' if x else 'Weekday')

# Категоризація напоїв
coffee_map = {
    'Latte': 'Coffee',
    'Americano': 'Coffee',
    'Cappuccino': 'Coffee',
    'Espresso': 'Coffee',
    'Flat White': 'Coffee',
    'Hot Chocolate': 'Chocolate',
    'Tea': 'Tea'
}
df['category'] = df['coffee_name'].map(coffee_map)

# --- 2️⃣ Аналітичні показники ---
# 1. Середній чек по пікових періодах
avg_check_peak = df.groupby('peak_period')['money'].mean().round(2)
print("Середній чек по пікових періодах:\n", avg_check_peak)

# 2. Кількість продажів по днях тижня та категоріях
sales_weekday_category = df.groupby(['weekday', 'category'])['money'].count().unstack().fillna(0)
print("\nПродажі по днях тижня та категоріях:\n", sales_weekday_category)

# 3. Продажі по типу платежу та пікових періодах
sales_cash_peak = df.groupby(['cash_type', 'peak_period'])['money'].count().unstack().fillna(0)
print("\nПродажі по типу платежу та пікових періодах:\n", sales_cash_peak)

# 4. Середній чек по днях тижня
avg_check_weekday = df.groupby('weekday')['money'].mean().round(2)
print("\nСередній чек по днях тижня:\n", avg_check_weekday)

# 5. ТОП-5 найпопулярніших напоїв
top_drinks = df['coffee_name'].value_counts().head(5)
print("\nТОП-5 найпопулярніших напоїв:\n", top_drinks)

# 6. Частка оплат карткою і готівкою
payment_share = df['cash_type'].value_counts(normalize=True).round(3) * 100
print("\nЧастка оплат за типом платежу (%):\n", payment_share)

# 7. Середній дохід по категоріях напоїв
avg_income_category = df.groupby('category')['money'].mean().round(2)
print("\nСередній чек по категоріях:\n", avg_income_category)

# 8. Загальна виручка по днях тижня
revenue_by_day = df.groupby('weekday')['money'].sum().round(2)
print("\nЗагальна виручка по днях тижня:\n", revenue_by_day)



Середній чек по пікових періодах:
 peak_period
Evening Peak    33.09
Lunch Peak      31.31
Morning Peak    30.78
Off-Peak        31.94
Name: money, dtype: float64

Продажі по днях тижня та категоріях:
 category   Chocolate  Coffee
weekday                     
Friday            45     304
Monday            35     324
Saturday          27     253
Sunday            40     223
Thursday          49     303
Tuesday           50     286
Wednesday         36     302

Продажі по типу платежу та пікових періодах:
 peak_period  Evening Peak  Lunch Peak  Morning Peak  Off-Peak
cash_type                                                    
card                  683         691           893      1280
cash                   16          13            29        31

Середній чек по днях тижня:
 weekday
Friday       31.72
Monday       31.95
Saturday     31.50
Sunday       31.93
Thursday     31.69
Tuesday      31.86
Wednesday    31.56
Name: money, dtype: float64

ТОП-5 найпопулярніших напоїв:
 coffee_name

Висновок:

Середній чек у вечірній піковий період (17:00–19:00) є найвищим — близько 33 грн, що свідчить про більшу готовність клієнтів витрачати після роботи.
Найнижчий чек спостерігається зранку (≈30.8 грн), коли переважають короткі замовлення “to go”.
В обідній час (12:00–14:00) чек помірний (≈31.3 грн).
У “непікові” години (Off-Peak) середній чек залишається стабільним (≈31.9 грн).
Найбільша кількість продажів припадає на будні дні (Monday–Friday), особливо у Thursday та Tuesday, коли активність клієнтів найвища.
У вихідні дні (Saturday–Sunday) спостерігається зниження кількості покупок, але частка “Chocolate” напоїв зростає — це може свідчити про більш релаксовані покупки на вихідних.
Більшість покупок (≈97%) оплачуються карткою, що підтверджує цифровізацію клієнтської поведінки.
Готівкові операції становлять незначну частку, найчастіше у пікові години — ймовірно, через черги або технічні збої з терміналами.
Середній чек у п’ятницю зазвичай трохи вищий — клієнти охочіше купують десерти або другий напій.
Понеділок має найнижчий середній чек — типова “кава, щоб прокинутись”.
Решта днів показує стабільну поведінку клієнтів.
У лідерах стабільно Latte та Americano — класичні напої, що формують основний трафік продажів.
Cappuccino та Espresso займають середні позиції, а Hot Chocolate частіше купують у вихідні.
Це підтверджує стандартну структуру попиту кав’ярень.
Карткові платежі складають понад 95% усіх операцій. Це зручно для бізнесу — менше ризиків, швидша обробка транзакцій та точніша аналітика.
Готівкові платежі — лише залишковий канал, можливо, через старших клієнтів або технічні причини.
Найдорожчі категорії — Chocolate напої, що мають більшу ціну порції.
Кава демонструє стабільний середній чек, але значно більший обсяг продажів, що робить її ключовим джерелом доходу.